In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam

from keras.optimizers.legacy import Adam as LegacyAdam

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pickle as pkl
from src.utils.data_transform import *
import pandas as pd
import os 
import pickle 
import json

In [2]:
def binary_cross_entropy_loss(y_true, y_pred):
    return -K.sum((y_true * K.log(y_pred)) - ((1 - y_true) * K.log(1 - y_pred)))

In [ ]:
def build_model():
    model = Sequential()
    model.add(LSTM(64, input_shape=(35, 5), recurrent_activation='hard_sigmoid', activation='tanh', return_sequences=True))
    model.add(LSTM(64, recurrent_activation='hard_sigmoid', activation='tanh'))
    model.add(Dense(1, activation='sigmoid'))
    model.add(Dropout(0.5))
    optimizer = RMSprop(learning_rate=1e-3)
    model.compile(loss=binary_cross_entropy_loss, optimizer=optimizer)
    
    return model

# στο training εχω true return sequences
# για prediction δε θα εχει true, 
# ----
# @prediction i use time distributed & return sequences true
# @training 
# 
# μπορω να εκπαιδευσω το end to end (convolution lstm το εκπαιδευω για μπουκιες)
# γνωριζω το micromecents 1.
# 
# ----
# @training != prediction lstm network
# time
# 
# εκπαιδευω με νορμαλ μπουκιες,
# 
# κανω prediciton με 3.5 sliding window


In [ ]:
#data loading/splitting etc
with open("../data/lstm_training_data/x_3.pkl", "rb") as f:
    x_train = pickle.load(f)
y_train = []

In [ ]:
with open("../data/FIC.pkl", "rb") as f:
    FIC = pickle.load(f)

In [ ]:
FIC.keys()

In [ ]:
lstm_label = FIC["bite_gt"]
print(lstm_label)

In [ ]:
mm_gt = FIC["mm_gt"]

In [ ]:
len(lstm_label[0])
# len(mm_gt[0])

In [ ]:
model.fit(x_train, y_train, epochs=6, batch_size=32)